In [6]:
#Import das bibliotecas
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [7]:
#Carrega a lista de Funcis a pesquisar
dados = pd.read_csv('input.csv')
dados.count()

CHV_FUN    32
dtype: int64

In [3]:
#Cria um dicionário para armazenar os resultados
dic_resultado = {'chave':[], 'total':[], 'grupo':[]}

In [4]:
#Instancia e configura um driver para o Chrome
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [5]:
#Navega para página do JogaJunto
url = 'https://jogajuntobb.com.br/'
driver.get(url)

Logar e navegar manualmente até a página da Rede JogaJunto

In [6]:
#Loop para percorrer todos os funcis a pesquisar
for index, row in dados.iterrows():
    valor = row['CHV_FUN']
    #Digita o valor da matricula e tecla enter (\n)
    driver.find_element(By.TAG_NAME, 'input').send_keys(valor+ "\n")
    time.sleep(6)
    #Clica no botão "VER MAIS"
    driver.find_elements(By.TAG_NAME, 'button')[1].click()
    time.sleep(6)
    #Clica no botão "Participação em realizações"
    driver.find_elements(By.TAG_NAME, 'button')[2].click()
    time.sleep(6)
    #Percorre todas as realizações, somando a pontuação total
    total = 0
    elementos = driver.find_elements(By.TAG_NAME, 'b')
    for elemento in elementos:
        total = total + int(elemento.text.replace('\nPONTOS','').replace('.',''))
    dic_resultado['chave'].append(valor)
    dic_resultado['total'].append(total)
    dic_resultado['grupo'].append(driver.find_elements(By.TAG_NAME, 'span')[1].text)
    #Clica no botão "Voltar" para reiniciar o loop
    driver.find_elements(By.TAG_NAME, 'button')[0].click()
    time.sleep(6)

ValueError: invalid literal for int() with base 10: 'Mandou Bem'

In [9]:
#Transforma o dicionário num dataframe pandas
result = pd.DataFrame(dic_resultado)

result.head(5)

,chave,total,grupo
0,F0169286,9600,Audit/Aud Sênior - Executiva 2
1,F0310564,5100,Audit/Aud Sênior - Executiva 2
2,F0443657,5100,Audit/Aud Sênior - Executiva 2
3,F1348799,4200,Audit/Aud Sênior - Executiva 2
4,F1765599,1800,DIVAR/GERENTE SETOR


In [10]:
result = result.sort_values(by=['total'], ascending=False)
result.to_csv('output.csv',sep = ';', encoding = 'latin-1', index=False)